##### Implementing Backpropagation From Scratch on Python 3+
- Let's see if theory and practice are the same thing.

In [4]:
# Provide some structure for our neural network
import numpy as np

class Network:
    def __init(self, structure):
        self.structure = structure
        self.num_layers = len(structure)
        self.Bn = [np.random.randn(l, 1) for l in structure[1:]]
        self.Wn = [np.random.randn(l, next_l) for l, next_l in zip(structure[:-1], structure[1:])]